In [ ]:
#%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


^C


  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git 'C:\Users\KEKE\AppData\Local\Temp\pip-req-build-qq67fr8m'


  Cloning https://github.com/unslothai/unsloth.git to c:\users\keke\appdata\local\temp\pip-req-build-qq67fr8m
  Resolved https://github.com/unslothai/unsloth.git to commit 575ef4d9c094d1c6d13f5414df4e7580347529ee
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for unsloth: filename=unsloth-2025.2.15-py3-none-any.whl size=189643 sha256=7efe0ca82afcf1212b70afbe6ca8c95c067047b5bc36ef66896c7f76f1f3839d
  Stored in directory: C:\Users\KEKE\AppData\Local\Temp\pip-ephem-wheel-cache-wi05awbt\wheels\60\3e\1f\e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.2.15
    Uninstalling unsloth-2025.2.15:


In [2]:
!pip install --upgrade transformers peft

In [ ]:
from huggingface_hub import login
#from google.colab import userdata

hf_token = '**********'

login(hf_token)

In [7]:
# 导入wandb库 - Weights & Biases，用于机器学习实验跟踪和可视化
import wandb

wb_token = userdata.get('WB_TOKEN')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B sentiment',    # 设置项目名称 - 这里是用于SQL分析的DeepSeek模型微调项目
    job_type="training",
    anonymous="allow"# 允许匿名访问 # "allow"表示即使没有wandb账号的用户也能查看这个项目
)

ModuleNotFoundError: No module named 'wandb'

In [3]:
# 从unsloth库中导入FastLanguageModel类
# unsloth是一个优化的语言模型加载和训练库
from unsloth import FastLanguageModel

# 设置模型参数
# 最大序列长度，即模型能处理的最大token数量
max_seq_length = 2048
dtype = None # 数据类型设置为None，让模型自动选择合适的数据类型
load_in_4bit = True # 启用4bit量化加载 # 4bit量化可以显著减少模型内存占用，但可能略微影响模型性能

# 加载预训练模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,     # 设置最大序列长度
    dtype = dtype,# 设置数据类型
    load_in_4bit = load_in_4bit,  # 启用4bit量化加载
    token = hf_token, # 使用Hugging Face的访问令牌
)

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
prompt = """Below is a comment. Analyze the sentiment of the comment directly.

### Comment:
Ooo duck congee my favourite. The oysters looked scrumptious but the pee pee soup, I’m sorry. Today’s video was great, thank you Amy.

### Response:
"""


In [ ]:
# 将模型切换到推理模式
FastLanguageModel.for_inference(model)
# 对输入文本进行分词和编码
# [prompt] - 将prompt放入列表中，因为tokenizer期望批处理输入
# return_tensors="pt" - 返回PyTorch张量格式
# to("cuda") - 将张量移动到GPU上进行计算
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# 使用模型生成响应
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 后处理生成的输出
# batch_decode - 将标记ID序列解码回文本
# split("### Response:")[1] - 提取"### Response:"之后的部分，即模型的实际回答
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


#微调前：模型能给出情感分析，但是太啰嗦




The comment begins with a positive expression of liking "duck congee," which is a traditional Chinese dish. The user then mentions oysters, describing them as "scrumptious," which further adds to the positive sentiment. However, the phrase "pee pee soup" is problematic. "Pee pee" is slang for urine, which is a bathroom-related term and not related to soup. This creates a confusing and potentially offensive context. Despite this, the overall sentiment seems positive because the user thanks the content creator, Amy, for the video, indicating that the problematic phrase might be a mistake or slip of the tongue. The user is likely expressing appreciation for the video content despite the confusing and inappropriate mention of "pee pee soup."

### Sentiment Analysis:
- Positive elements: Expressing enjoyment for duck congee and oysters, gratitude towards Amy.
- Negative elements: The mention of "pee pee soup," which is inappropriate and could be confusing or offensive.
- Overall Sentiment:

In [ ]:
train_prompt_style = """Below is an instruction that describes a task,
paired with an input that provides further context.
Write a response directly without any thinking.

### Instruction:
You are a sentiment expert with advance understanding of video comments.
You can understand sentiment from the comment.
Think like sentiment expert and generate result of the comment.

### Comment:
{}

### Response:
{}"""


In [ ]:

from datasets import load_dataset
# 使用公开的数据源，结果中仅包含（neg|pos|nut)
ds = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
# 检查数据
print(ds['validation'][:1])
dataset = ds['test'][:25]
print(dataset['text'][:1])
print(dataset['label'][:1])
print(dataset['sentiment'][:1])

{'id': [317], 'text': ['Laying in bed til workkk... Oh the life. Definitely pinched a nerve.'], 'label': [0], 'sentiment': ['negative']}
['getting cds ready for tour']
[1]
['neutral']


In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# 风格化，用于训练调整
def format_prompt(examples):
    inputs = examples["text"] # 使用 text 作为输入
    outputs = examples["sentiment"] # 使用 sentiment 作为输出

    texts = []
    for input, output in zip(inputs, outputs):
        text = train_prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return texts



In [ ]:
FastLanguageModel.for_training(model)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,   # LoRA的秩(rank)值，决定了低秩矩阵的维度，较大的r值(如16)可以提供更强的模型表达能力，但会增加参数量和计算开销，较小的r值(如4或8)则会减少参数量，但可能影响模型性能，通常在4-16之间选择,需要在性能和效率之间权衡
    target_modules=[ #指定需要应用LoRA微调的模块列表，q_proj, k_proj, v_proj: 注意力机制中的查询、键、值投影层
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj", #注意力输出投影层
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16, #缩放参数，用于控制LoRA更新的强度，通常设置为与r相同的值，较大的alpha会增加LoRA的影响力，较小的alpha则会减弱LoRA的影响
    lora_dropout=0, #LoRA层的dropout率，0表示不使用dropout，增加dropout可以帮助防止过拟合，但可能影响训练稳定性，在微调时通常设为0或很小的值
    bias="none", #是否微调偏置项，"none"表示不微调偏置参数，也可以设置为"all"或"lora_only"来微调不同范围的偏置
    use_gradient_checkpointing="unsloth",  # 梯度检查点策略，"unsloth"是一种优化的检查点策略，适用于长上下文可以显著减少显存使用，但会略微增加计算时间对处理长文本特别有用
    random_state=3407, #随机数种子，控制初始化的随机性，固定种子可以确保实验可重复性
    use_rslora=False, #是否使用RSLoRA(Rank-Stabilized LoRA) False表示使用标准LoRARSLoRA是一种改进的LoRA变体，可以提供更稳定的训练
    loftq_config=None, #LoftQ配置None表示不使用LoftQ量化LoftQ是一种用于模型量化的技术，可以减少模型大小
)


Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
print(dataset)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, Trainer
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    formatting_func=format_prompt,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    ## 训练参数配置
    args=TrainingArguments(
        # 批处理相关
        per_device_train_batch_size=2, # 每个设备（GPU）的训练批次大小
        gradient_accumulation_steps=4,# 梯度累积步数，用于模拟更大的批次大小
         # 训练步数和预热
        warmup_steps=5,# 学习率预热步数，逐步增加学习率
        max_steps=60,# 最大训练步数
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(), # 如果不支持 bfloat16，则使用 float16
        bf16=is_bfloat16_supported(),# 如果支持则使用 bfloat16，通常在新型 GPU 上性能更好
        logging_steps=10,# 每10步记录一次日志
        optim="adamw_8bit", # 使用8位精度的 AdamW 优化器
        weight_decay=0.01,# 权重衰减率，用于防止过拟合
        lr_scheduler_type="linear",# 学习率调度器类型，使用线性衰减
        seed=3407,# 随机种子，确保实验可重复性
        output_dir="outputs", # 模型和检查点的输出目录
    ),
)


Applying formatting function to train dataset (num_proc=2):   0%|          | 0/31232 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/31232 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/31232 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/31232 [00:00<?, ? examples/s]

Applying formatting function to eval dataset (num_proc=2):   0%|          | 0/5206 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/5206 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/5206 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 31,232 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,3.213400
20,1.153400
30,1.016600
40,1.143900
50,1.048100
60,1.190700


In [ ]:
# 定义提示模板
prompt_style = """Below is an instruction that describes a task,
paired with an input that provides further context.
Write a response directly without any thinking.

### Instruction:
You are a sentiment expert with advance understanding of video comments.
You can understand sentiment from the comment.
Think like sentiment expert and generate result of the comment.

### Comment:
{}

### Response:
{}"""

# 定义测试用的 text
comment1 = """
Ooo duck congee my favourite. The oysters looked scrumptious but the pee pee soup, I’m sorry. Today’s video was great, thank you Amy.
"""

# 将模型设置为推理模式
FastLanguageModel.for_inference(model)
# 准备输入数据
# 使用提示模板格式化查询，并转换为模型可处理的张量格式
inputs = tokenizer([prompt_style.format(comment1, "")], return_tensors="pt").to("cuda")

# 生成响应
outputs = model.generate(
    input_ids=inputs.input_ids, # 输入的标记ID
    attention_mask=inputs.attention_mask, # 注意力掩码，用于处理填充
    max_new_tokens=1200, # 最大生成的新标记数
    use_cache=True,  # 使用缓存以提高生成速度
)
# 解码模型输出
# 使用分词器将输出转换回文本，并提取Response部分
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

#微调后：得到准确且精炼的分析结果



positive<｜end▁of▁sentence｜>


In [ ]:
local_path="deepseek_nlp_model2"

model.save_pretrained(local_path)
tokenizer.save_pretrained(local_path)

# Save merged model
# model.save_pretrained_merged(local_path, tokenizer, save_method="merged_16bit")


('deepseek_nlp_model2/tokenizer_config.json',
 'deepseek_nlp_model2/special_tokens_map.json',
 'deepseek_nlp_model2/tokenizer.json')

In [ ]:
model.push_to_hub(local_path)
tokenizer.push_to_hub(local_path)


README.md:   0%|          | 0.00/627 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/deepseek_nlp_model2


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = "/content/drive/MyDrive/deepseek_nlp_model2"
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/deepseek_nlp_model2/tokenizer_config.json',
 '/content/drive/MyDrive/deepseek_nlp_model2/special_tokens_map.json',
 '/content/drive/MyDrive/deepseek_nlp_model2/tokenizer.json')

In [ ]:
import pandas as pd


# Load the dataset
comments = pd.read_csv("/content/drive/MyDrive/video_comments.csv")

# Print some info (optional)
print(comments[:5])


   Unnamed: 0                                              title  \
0           0  Shaxian: The Chinese snack capital you’ve neve...   
1           1  Shaxian: The Chinese snack capital you’ve neve...   
2           2  Shaxian: The Chinese snack capital you’ve neve...   
3           3  Shaxian: The Chinese snack capital you’ve neve...   
4           4  Shaxian: The Chinese snack capital you’ve neve...   

                                         description          publishDate  \
0  I finally visited Shaxian 沙县! The birthplace o...  2022-09-13T09:33:18   
1  I finally visited Shaxian 沙县! The birthplace o...  2022-09-13T09:33:18   
2  I finally visited Shaxian 沙县! The birthplace o...  2022-09-13T09:33:18   
3  I finally visited Shaxian 沙县! The birthplace o...  2022-09-13T09:33:18   
4  I finally visited Shaxian 沙县! The birthplace o...  2022-09-13T09:33:18   

   viewCount  commentCount  likes  dislikes          commentDate  \
0     182843           399   6175         0  2022-09-13T09:3

In [ ]:
from tqdm import tqdm
import time
import pandas as pd

# Assuming 'comments' DataFrame is already loaded and processed as in the original code

# Process each formatted comment with tqdm progress bar
for index, comment in tqdm(comments.iterrows(), total=len(comments), desc="Analyzing sentiments"):
    inputs = tokenizer([prompt_style.format(comment['commentText'], "")], return_tensors="pt").to("cuda")
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1200,
        use_cache=True,
    )
    response = tokenizer.batch_decode(outputs)
    comments.at[index,'sentiment'] = response[0].split("### Response:")[1].split("<｜end▁of▁sentence｜>")[0] # Extract sentiment directly

comments.to_csv("/content/drive/MyDrive/video_comments_sentiment_0217.csv")


Analyzing sentiments:   0%|          | 224/49039 [01:19<4:47:41,  2.83it/s]


KeyboardInterrupt: 